In [1]:
import pm4py

file_path ="./datasets/event_logs/Helpdesk.xes"

log = pm4py.read_xes(file_path)

parsing log, completed traces :: 100%|██████████| 4580/4580 [00:01<00:00, 4154.54it/s]


In [2]:
flattern_log: list[dict[str, any]] = ([{**event,
                                                'caseid': trace.attributes['concept:name']}
                                               for trace in log for event in trace])


In [3]:
import pandas as pd

df = pd.DataFrame(flattern_log)


In [4]:
df["name_and_transition"] = df["concept:name"] + \
            "_" + df["lifecycle:transition"]
df = df[['time:timestamp', 'name_and_transition', "caseid"]]

In [5]:
df

,time:timestamp,name_and_transition,caseid
0,2012-10-09 14:50:17+03:00,Assign seriousness_complete,Case1
1,2012-10-09 14:51:01+03:00,Take in charge ticket_complete,Case1
2,2012-10-12 15:02:56+03:00,Take in charge ticket_complete,Case1
3,2012-10-25 11:54:26+03:00,Resolve ticket_complete,Case1
4,2012-11-09 12:54:39+02:00,Closed_complete,Case1
...,...,...,...
21343,2010-09-02 10:11:00+03:00,Closed_complete,Case4579
21344,2012-01-03 09:33:43+02:00,Take in charge ticket_complete,Case4580
21345,2012-01-10 15:30:11+02:00,Wait_complete,Case4580
21346,2012-01-10 17:07:40+02:00,Resolve ticket_complete,Case4580


In [6]:
from Utils.Constants import Constants

from datetime import timedelta

newData = list() 
for case, group in df.groupby('caseid'):
    group.sort_values("time:timestamp", ascending=True, inplace=True)
    strating_time = group.iloc[0]["time:timestamp"] - \
        timedelta(microseconds=1)
    ending_time = group.iloc[-1]["time:timestamp"] + \
        timedelta(microseconds=1)
    traces = group.to_dict('records')

    # Add start and end tags.
    traces.insert(
        0, {"caseid": case, "time:timestamp": strating_time, "name_and_transition": Constants.SOS_VOCAB})
    traces.append(
        {"caseid": case, "time:timestamp": ending_time, "name_and_transition": Constants.EOS_VOCAB})
    newData.extend(traces)

df = pd.DataFrame(newData)
df['name_and_transition'] = df['name_and_transition'].astype(
    'category')

In [7]:
df

,caseid,time:timestamp,name_and_transition
0,Case1,2012-10-09 14:50:16.999999+03:00,<SOS>
1,Case1,2012-10-09 14:50:17+03:00,Assign seriousness_complete
2,Case1,2012-10-09 14:51:01+03:00,Take in charge ticket_complete
3,Case1,2012-10-12 15:02:56+03:00,Take in charge ticket_complete
4,Case1,2012-10-25 11:54:26+03:00,Resolve ticket_complete
...,...,...,...
30503,Case999,2013-02-12 16:06:37+02:00,Assign seriousness_complete
30504,Case999,2013-02-25 11:37:20+02:00,Take in charge ticket_complete
30505,Case999,2013-03-14 16:24:30+02:00,Resolve ticket_complete
30506,Case999,2013-03-29 16:24:45+02:00,Closed_complete


In [8]:
vocab_dict: dict[str, int] = {}
for i, cat in enumerate(df['name_and_transition'].cat.categories):
            # plus one, since we want to remain "0" for "<PAD>"
    vocab_dict[cat] = i+1
vocab_dict[Constants.PAD_VOCAB] = 0

In [9]:
df['cat'] = df['name_and_transition'].apply(lambda c: vocab_dict[c])

############ Create the df only consisted of trace and caseid ############
final_df_data: list[dict[str, any]] = []
for caseid, group in df.groupby('caseid'):
    final_df_data.append({
        "trace": list(group['cat']),
        "caseid": caseid
    })

In [10]:
pd.DataFrame(final_df_data)

,trace,caseid
0,"[2, 3, 14, 14, 12, 4, 1]",Case1
1,"[2, 3, 14, 12, 4, 1]",Case10
2,"[2, 3, 14, 10, 12, 4, 1]",Case100
3,"[2, 3, 3, 14, 12, 4, 1]",Case1000
4,"[2, 3, 14, 12, 4, 1]",Case1001
...,...,...
4575,"[2, 3, 14, 16, 14, 12, 4, 1]",Case995
4576,"[2, 3, 14, 12, 4, 1]",Case996
4577,"[2, 3, 14, 12, 4, 1]",Case997
4578,"[2, 3, 14, 16, 12, 4, 1]",Case998
